In [10]:
# Import libraries and Packages
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime as dt
from datetime import datetime,tzinfo
from pytz import timezone
import time
import pytz
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pprint as pp


## Filter news by marker
This notebook is intended to perform the following processes:

    1.1 Creates marks for mapping statistically significant price changes to news articles within a windown of time

___

In [5]:
marker= pd.read_csv('marker.csv')
marker.head()
mark=marker['Date']
mark.head(5)

0    2018-02-04
1    2018-02-05
2    2018-02-06
3    2018-02-07
4    2018-02-08
Name: Date, dtype: object

__Modify datestamp for mapping purposes__

In [11]:
# create list of timestamps according to given datetime parameters
array_mark = []
for i in range(len(mark)):
    date = datetime.strptime(mark[i],'%Y-%m-%d')
    array_mark.append(date)
pp.pprint(array_mark[0:5])

[datetime.datetime(2018, 2, 4, 0, 0),
 datetime.datetime(2018, 2, 5, 0, 0),
 datetime.datetime(2018, 2, 6, 0, 0),
 datetime.datetime(2018, 2, 7, 0, 0),
 datetime.datetime(2018, 2, 8, 0, 0)]


__Allows for a given date range of consideration (+- N days from statistically significant price change)__

In [12]:
N = 3
days_before=[]
days_after=[]
for i in range(len(mark)):
    date = datetime.strptime(mark[i],'%Y-%m-%d')
    days_before.append((date-timedelta(days=N)).isoformat())
    days_after.append((date+timedelta(days=N)).isoformat())

In [18]:
# appends additional days to list -- used for mapping of articles
for i in range(len(mark)):
    date = datetime.strptime(mark[i],'%Y-%m-%d')
    array_mark.append(date-timedelta(days=1))
    array_mark.append(date+timedelta(days=1))
    array_mark.append(date-timedelta(days=2))
    array_mark.append(date+timedelta(days=2))
    array_mark.append(date-timedelta(days=3))
    array_mark.append(date+timedelta(days=3))
pp.pprint(array_mark[:10])

[datetime.datetime(2018, 2, 4, 0, 0),
 datetime.datetime(2018, 2, 5, 0, 0),
 datetime.datetime(2018, 2, 6, 0, 0),
 datetime.datetime(2018, 2, 7, 0, 0),
 datetime.datetime(2018, 2, 8, 0, 0),
 datetime.datetime(2018, 2, 9, 0, 0),
 datetime.datetime(2018, 2, 10, 0, 0),
 datetime.datetime(2018, 2, 11, 0, 0),
 datetime.datetime(2018, 2, 12, 0, 0),
 datetime.datetime(2018, 2, 13, 0, 0)]


In [21]:
# 1x8 array of 1's
np.ones(8)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

__Mark 'significant' dates__

In [40]:
# Marking of dates with statistically significant price changes
date_mark=pd.DataFrame({'Date':array_mark,
                       'Mark':np.ones(len(array_mark))})
pp.pprint(date_mark.head())
date_mark.to_csv('marked_dates.csv')

        Date  Mark
0 2018-02-04   1.0
1 2018-02-05   1.0
2 2018-02-06   1.0
3 2018-02-07   1.0
4 2018-02-08   1.0


In [46]:
## checking variable types ##
#type(date_mark['Date'][0])  # .Timestamp
#type(date_mark['Mark'][0]) # float type

__Prepare articles for marking__

In [27]:
df= pd.read_csv('short')

In [28]:
#Convert Timestamp into separate Date and Time
df['Date'] = pd.to_datetime(df['timeStamp']).dt.date

del df['timeStamp'] #Delete original datetime column
df.head()

,Unnamed: 0,author,contents,description,publisher,source_url,title,Date
0,0,Phillip Molnar,A British real estate company Monday launched ...,A British real estate company Monday launched ...,Sandiegouniontribune.com,http://www.sandiegouniontribune.com/business/r...,"Purplebricks, flat fee real estate listers, la...",2018-01-09
1,1,NaN,2018-01-09 — bbc.com\t\n\t\nsource article |...,``In parts of the continent - especially comme...,NaN,NaN,Why African millennials can't get enough of Bi...,2018-01-09
2,2,"Associated Press, By Associated Press","NEW YORK — Kodak, which traces its roots to th...","NEW YORK — Kodak, which traces its roots to th...",Bostonherald.com,http://www.bostonherald.com/news/national/2018...,Kodak surges at it becomes latest 'cryptocurre...,2018-01-09
3,3,NaN,2018-01-09 — thehill.com\t\n\t\nsource artic...,``JPMorgan Chase CEO Jamie Dimon has walked ba...,NaN,NaN,"Dimon: ""I regret calling bitcoin a fraud""",2018-01-09
4,4,NaN,2018-01-09 — reuters.com\t\n\t\nsource artic...,"``Staff at the regulatory agency ""expressed co...",NaN,NaN,Fund managers say bitcoin ETF proposals withdr...,2018-01-09


__Additional Pre-Processing__

In [33]:
# datatype coercion of features
df.Date = df.Date.astype(str)               
date_mark.Date = date_mark.Date.astype(str)

__Mark articles according to statistically significant price changes__

In [35]:
marked = pd.merge(df,date_mark, how='left', left_on='Date',right_on='Date')

In [36]:
# handle missing marks
marked.Mark = marked.Mark.fillna(0)

In [37]:
marked.head()

,Unnamed: 0,author,contents,description,publisher,source_url,title,Date,Mark
0,0,Phillip Molnar,A British real estate company Monday launched ...,A British real estate company Monday launched ...,Sandiegouniontribune.com,http://www.sandiegouniontribune.com/business/r...,"Purplebricks, flat fee real estate listers, la...",2018-01-09,0.0
1,1,NaN,2018-01-09 — bbc.com\t\n\t\nsource article |...,``In parts of the continent - especially comme...,NaN,NaN,Why African millennials can't get enough of Bi...,2018-01-09,0.0
2,2,"Associated Press, By Associated Press","NEW YORK — Kodak, which traces its roots to th...","NEW YORK — Kodak, which traces its roots to th...",Bostonherald.com,http://www.bostonherald.com/news/national/2018...,Kodak surges at it becomes latest 'cryptocurre...,2018-01-09,0.0
3,3,NaN,2018-01-09 — thehill.com\t\n\t\nsource artic...,``JPMorgan Chase CEO Jamie Dimon has walked ba...,NaN,NaN,"Dimon: ""I regret calling bitcoin a fraud""",2018-01-09,0.0
4,4,NaN,2018-01-09 — reuters.com\t\n\t\nsource artic...,"``Staff at the regulatory agency ""expressed co...",NaN,NaN,Fund managers say bitcoin ETF proposals withdr...,2018-01-09,0.0


In [38]:
marked.to_csv('marked_news_short.csv')

___